In [34]:
import boto3
import pandas as pd

In [35]:
apig_client = boto3.client('apigateway')
ddb_client = boto3.client('dynamodb')
idp_client = boto3.client('cognito-idp')

API_NAME_TO_USAGE_PLAN = {
    'Baseline EMV': '170hjm',
    'Vertical EMV': '170hjm',
    'Bot Detector': 'b9k8q4',
    'Bot Detection': 'b9k8q4',
    'Knowledge Graph Evaluator': 'qcl01z',
    'Knowledge Graph': 'qcl01z',
    'Enhanced Personality Evaluator': 'ue56mc',
    'Personality Graph': 'ue56mc',
    'Sentiment Analyzer':'976kf0',
    'Targeted Sentiment': '976kf0',
    'Object Recognition':'usoanm',
    'Social Post Image Detector': 'usoanm'
}

USER_POOL_ID = 'us-west-2_17Cxl3ix7'
REQUEST_ACCESS_TABLE = 'demo-inquiries'
FED_ID_TO_USER_ID_SUB_TABLE = 'customer_to_api_key_association'

In [36]:
def get_all_users():
    '''
    Returns a list of all users for a given cognito user pool
    '''
    response = idp_client.list_users(
        UserPoolId=USER_POOL_ID,
        Limit=10,
    )
    return response['Users']

def get_user_by_sub(sub):
    '''
    Returns a user if found for a given sub (unique identifier)
    '''
    filt = "sub='{}'".format(sub)
    try:
        response = idp_client.list_users(
            UserPoolId=USER_POOL_ID,
            Limit=10,
            Filter=filt
        )
    except Exception as e:
        print(e)
        
    return response

def get_username(sub):
    '''
    Parse username from user record
    '''
    return get_user_by_sub(sub)['Users'][0]['Username']

#### Helpers to pull data from DynamoDB tables

In [37]:
def get_demo_access_requests():
    '''
    Get list of requests for demo access
    '''
    response = ddb_client.scan(
        TableName=REQUEST_ACCESS_TABLE,
        Limit=100
    )
    return response['Items']

def get_id_mappings():
    '''
    Get list of associations between federated identities and user pool identities
    '''
    response = ddb_client.scan(
        TableName=FED_ID_TO_USER_ID_SUB_TABLE,
        Limit=100
    )
    return response['Items']

In [38]:
def ddb_records_to_list_of_dicts(objs):
    '''
    Dynamo sucks.  This maps keys to values, removing the additional dict specifying data type.
    '''
    obj_list = []
    for obj in objs:
        u = {}
        for k, v in obj.items():
            for value in v.values():
                u[k]=value
        obj_list.append(u)
    return obj_list
        

#### Get a list Cognito Identity ID's mapped to Cognito User Pool Usernames

In [39]:
user_list = ddb_records_to_list_of_dicts(get_id_mappings())
u_df = pd.DataFrame(user_list)
u_df = u_df.assign(username=u_df['UserId'].apply(get_username))

Print Data Frame

In [40]:
u_df

,ApiKeyId,Id,UserId,UserPoolId,username
0,fvhv3hhwpi,us-west-2:e605018e-51a3-48ee-a8cd-1b9903fd528f,35b1ca2f-b95d-4225-b94f-f139eca1a916,placeholder,richardm@moorecommgroup.com
1,k15wh52cm4,us-west-2:aa7af9b0-ece3-426d-ac11-aee4403c497f,a1b3d127-2de3-42cd-a842-8abbb4086c61,placeholder,ytan@ayzenberg.com
2,q56wh54mn4,us-west-2:4469bf53-8ad1-40ce-97d4-a6076328e9e1,4e0dc03f-1c86-4248-98f5-06148e36ac3e,placeholder,smondelli@juliusworks.com
3,7ac05cgbtj,us-west-2:473a628a-beb0-41c3-9c3d-914e10c21537,9e01dfee-8a3e-4903-a700-0253e0a4cc99,placeholder,luca.rimoldi@9024sports.com
4,m3a08by512,us-west-2:c3bf1073-ff60-4f1e-b1cf-b0793ecbc8b3,c6ae17ed-0858-43c5-b8d1-32e27260d2db,placeholder,jmoore@ayzenberg.com
5,yx3ongg780,us-west-2:76be6a4c-2c7b-4c3c-8d63-75eb02e02276,63e9cf9f-8066-41bb-842d-cb79b25b97f2,placeholder,sometester@test.com
6,vhsqb6msqj,us-west-2:96c45dd9-aab2-4ce8-af69-11a532bf32e4,55ca586e-c7b5-48cb-9ec5-cc22feda2bdd,placeholder,bhavika.rambhai@tnz.govt.nz
7,gke5j522x3,us-west-2:e14e4a27-5b2d-4fe9-adc6-36961d944350,b9abe05b-653f-4d37-8d58-747baae770c8,placeholder,asaldana@ayzenberg.com
8,ig4txz9v28,us-west-2:02699909-cabd-401d-8ecd-1a211783fa1b,316ba772-e73f-40d9-9f99-bec2f8c5014a,placeholder,test@test.com
9,565olg8p0c,us-west-2:405ea40f-29da-48de-9b44-c0e80244d877,2ca3550c-b995-4905-b33c-8f27d293b178,placeholder,thomas@edgeskills.com


In [19]:
iqs = get_demo_access_requests()

In [20]:
dr_df = pd.DataFrame(ddb_records_to_list_of_dicts(iqs))

In [21]:
dr_df

,email,requested_apis,requested_at
0,rkcudjoe@hookops.com,[Sentiment Analyzer],2018-01-18 17:49:10
1,luca.rimoldi@9024sports.com,"[Baseline EMV, Bot Detector, Enhanced Personal...",2018-01-18 11:57:10
2,bhavika.rambhai@tnz.govt.nz,"[Baseline EMV, Bot Detection, Knowledge Graph,...",2018-01-18 00:19:09
3,brandon@grin.co,"[Baseline EMV, Bot Detection, Knowledge Graph,...",2018-01-18 00:16:35
4,thomas@edgeskills.com,"[Knowledge Graph, Object Recognition, Personal...",2018-01-18 00:14:54
5,kim.harris@8451.com,"[Baseline EMV, Enhanced Personality Evaluator,...",2018-01-18 12:43:09


In [22]:
dr_df['requested_apis'][4]

['Knowledge Graph',
 'Object Recognition',
 'Personality Graph',
 'Targeted Sentiment']

In [44]:
for _, row in dr_df.iterrows():
    usage_plans = [ API_NAME_TO_USAGE_PLAN[r] for r in row['requested_apis']]
    email = row['email']
    
    if email == 'luca.rimoldi@9024sports.com':
        for up in usage_plans:
            subscribe_to_api(email, up)

{'ResponseMetadata': {'RequestId': 'b7edf01d-0069-11e8-af48-bbab3d2be271', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Tue, 23 Jan 2018 18:17:49 GMT', 'content-type': 'application/json', 'content-length': '93', 'connection': 'keep-alive', 'x-amzn-requestid': 'b7edf01d-0069-11e8-af48-bbab3d2be271'}, 'RetryAttempts': 0}, 'id': '7ac05cgbtj', 'type': 'API_KEY', 'name': 'us-west-2:473a628a-beb0-41c3-9c3d-914e10c21537'}
{'ResponseMetadata': {'RequestId': 'b842c622-0069-11e8-9757-e150e17fb129', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Tue, 23 Jan 2018 18:17:50 GMT', 'content-type': 'application/json', 'content-length': '93', 'connection': 'keep-alive', 'x-amzn-requestid': 'b842c622-0069-11e8-9757-e150e17fb129'}, 'RetryAttempts': 0}, 'id': '7ac05cgbtj', 'type': 'API_KEY', 'name': 'us-west-2:473a628a-beb0-41c3-9c3d-914e10c21537'}
{'ResponseMetadata': {'RequestId': 'b89e05ab-0069-11e8-b53f-dd17562d11fb', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Tue, 23 Jan 2018 18:17:50 GMT', '

In [43]:
def subscribe_to_api(email, usage_plan_id):
    
    row = u_df.loc[u_df['username'] == email ]
    api_key_id = list(row['ApiKeyId'])[0]
    try:
        resp = apig_client.create_usage_plan_key(
            usagePlanId=usage_plan_id,
            keyId=api_key_id,
            keyType="API_KEY"
        )
    except Exception as e:
        resp = e
    print(resp)

In [33]:
subscribe_to_api('luca.rimoldi@9024sports.com','170hjm')

NameError: name 'subscribe_to_api' is not defined

In [29]:
def get_associated_usage_plans(api_key):
    try:
        resp = apig_client.get_usage_plans(
            keyId=api_key
        )
    except Exception as e:
        print(e)
        return
    return resp['items']

In [30]:
get_associated_usage_plans('h3kk80mwrb')

[{'apiStages': [{'apiId': 'sg4a47l6w6', 'stage': 'dev'}],
  'description': 'This is the basic knowledge graph usage plan',
  'id': 'qcl01z',
  'name': 'Basic Knowledge Graph',
  'quota': {'limit': 10000, 'offset': 0, 'period': 'MONTH'},
  'throttle': {'burstLimit': 300, 'rateLimit': 100.0}},
 {'apiStages': [{'apiId': 'ibfxs3d765', 'stage': 'dev'},
   {'apiId': 'qfqw8rsdnj', 'stage': 'dev'},
   {'apiId': 'ra2k43k3b8', 'stage': 'dev'}],
  'description': 'Free-Tier access to the sentiment analysis API for promotional purposes.',
  'id': '976kf0',
  'name': 'Basic Seneval Usage Plan',
  'quota': {'limit': 10000, 'offset': 0, 'period': 'DAY'},
  'throttle': {'burstLimit': 100, 'rateLimit': 10.0}},
 {'apiStages': [{'apiId': 'zjflm652a6', 'stage': 'Dev'}],
  'description': 'Object Detection Usage plan',
  'id': 'usoanm',
  'name': 'Basic Object Detection',
  'quota': {'limit': 200000, 'offset': 0, 'period': 'MONTH'},
  'throttle': {'burstLimit': 500, 'rateLimit': 200.0}},
 {'apiStages': [{'ap

In [31]:
def find_by_sub(sub):
    data = ddb_client.query(
        TableName='customer_to_api_key_association',
        IndexName='cognito_user_pool_id_index',
        KeyConditionExpression="UserId = :sub",
        ExpressionAttributeValues={
            ":sub": {"S": sub}
        }
    )
    item = ddb_records_to_list_of_dicts(data['Items'])[0]
    return item

In [32]:
find_by_sub('811e5819-e33d-4cbd-9dab-3f8f86eaa132')

{'ApiKeyId': 'h3kk80mwrb',
 'Id': 'us-west-2:16431038-f615-4368-8c08-0aacf715c94c',
 'UserId': '811e5819-e33d-4cbd-9dab-3f8f86eaa132',
 'UserPoolId': 'placeholder'}

In [5]:
import boto3

client = boto3.client('apigateway')

key_data = client.get_api_key(apiKey='ybicl4oc1b',includeValue=True)

key_data['value'] # API Key Value